In [156]:
import pandas as pd
from pymongo import MongoClient, GEO2D

In [157]:
client = MongoClient('mongodb://localhost:27017/')
db = client.offices
datos_mongo = db.offices


puntuaciones= {
    'video_juegos' : 100,
    'software' : 50,
    'big_company' : 25,
    'startup' : 40
    }

In [158]:
# Obtener datos de mongo con el indice 2d geo.
def get_geo_data(datos):
    cursor_geo= datos.find()
    df = pd.DataFrame(cursor_geo)
    df = df.drop('_id', axis=1)
    return df

In [159]:
'''def get_offices_near(longitud, latitud):
    df_location = pd.DataFrame(datos_mongo.find({
                                "2Dgeo": {
                                 "$near": {
                                   "$geometry": {
                                      "type": "Point" ,
                                      "coordinates": [ longitud, latitud ]
                                   },
                                   "$maxDistance": 2000, # In meters
                                 }
                               }
                            }))
    return df_location.drop('_id', axis=1)'''
    

'def get_offices_near(longitud, latitud):\n    df_location = pd.DataFrame(datos_mongo.find({\n                                "2Dgeo": {\n                                 "$near": {\n                                   "$geometry": {\n                                      "type": "Point" ,\n                                      "coordinates": [ longitud, latitud ]\n                                   },\n                                   "$maxDistance": 2000, # In meters\n                                 }\n                               }\n                            }))\n    return df_location.drop(\'_id\', axis=1)'

In [179]:
def sum_points(reg):
    points = 0
    if str(reg.category_code) == 'games_video': points+= 100
    if str(reg.category_code) == 'software': points+= 50
    if str(reg.type) == 'Startup': points+= 40
    if str(reg.type) == 'BigCompany': points+= 25
    return points

def get_offices_near(reg):
    points= list()
    lat = float(reg.latitude)
    long = float(reg.longitude)
    df_location = pd.DataFrame(datos_mongo.find({
                                "2Dgeo": {
                                 "$near": {
                                   "$geometry": {
                                      "type": "Point" ,
                                      "coordinates": [ long, lat ]
                                   },
                                   "$maxDistance": 3000, # In meters
                                 }
                               }
                            }))
    
    #points= len(df_location)
    #print('enviar', reg.name)
    points.append(df_location.apply(sum_points, axis=1))
    return sum(points[0])

In [180]:
def calculate_points(df):
    df['points'] = df.apply(get_offices_near, axis=1)
    return df

In [183]:
#Main
df_geo_mongo = get_geo_data(datos_mongo)
df_points= calculate_points(df_geo_mongo)
df_points= df_points.sort_values('points', ascending=False)

#TODO pintar el punto en el mapa.

In [184]:
df_points

,2Dgeo,category_code,city,country_code,founded_year,latitude,longitude,name,number_of_employees,type,points
389,"[-122.421406, 37.781386]",games_video,San Francisco,USA,2007,37.781386,-122.421406,TubeSpree,11,BigCompany,4215
165,"[-122.409173, 37.788482]",games_video,San Francisco,USA,2006,37.788482,-122.409173,Kyte,40,BigCompany,4090
7,"[-122.4084994, 37.7909427]",software,San Francisco,USA,2007,37.790943,-122.408499,Dropbox,350,BigCompany,4090
477,"[-122.407709, 37.781754]",software,San Francisco,USA,2007,37.781754,-122.407709,PX Interactive,8,BigCompany,4015
474,"[-122.407709, 37.781754]",software,San Francisco,USA,2007,37.781754,-122.407709,PX Interactive,8,BigCompany,4015
301,"[-122.423132, 37.792303]",software,San Francisco,USA,2008,37.792303,-122.423132,Tivix,18,Startup,3890
48,"[-122.419204, 37.775196]",software,San Francisco,USA,2008,37.775196,-122.419204,GitHub,106,BigCompany,3860
825,"[-122.419204, 37.775196]",software,San Francisco,USA,2008,37.775196,-122.419204,148Apps,1,Startup,3860
770,"[-122.419204, 37.775196]",games_video,San Francisco,USA,2008,37.775196,-122.419204,stickychicken,2,Startup,3860
659,"[-122.4194155, 37.7749295]",software,San Francisco,USA,2007,37.774929,-122.419415,Scripped,4,BigCompany,3860
